# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# pip install gmaps
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
#api_keys = "../api_keys.py"
#api_keys = "C:\\Users\\joeki\\Desktop\\python-api-challenge\\api_keys.py"
# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
weather_data = pd.read_csv("../output_data/cities.csv")
weather_data.head()

,City ID,City,Lat,lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,lagoa,39.0500,-27.9833,59.99,91,100,25.81,PT,1612659139
1,1,hilo,19.7297,-155.0900,78.80,47,40,9.22,US,1612659154
2,2,orocue,4.7912,-71.3354,84.15,57,69,8.14,CO,1612659435
3,3,torres,-29.3353,-49.7269,69.06,83,100,3.83,BR,1612659435
4,4,rikitea,-23.1203,-134.9692,78.78,68,45,9.57,PF,1612659322


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Configure gmaps
gmaps.configure(api_key=g_key)

locations = weather_data[["Lat", "lng"]].astype(float)

humidity = weather_data["Humidity"].astype(float)

In [4]:
# Plot Heatmap
fig = gmaps.figure(center=(0.0, 0.0), zoom_level=2)
max_intensity = np.max(humidity)

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights = humidity, dissipating=False, max_intensity=100, point_radius=3)

# Add layer
fig.add_layer(heat_layer)
#plt.savefig("C:/Users/joeki/Desktop/python-api-challenge/output_data/heatmap.png")

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

<Figure size 432x288 with 0 Axes>

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
narrow_city_df = weather_data.loc[(weather_data["Max Temp"] <=80) & (weather_data["Max Temp"] >=70) &
    (weather_data["Wind Speed"] <=10) & (weather_data["Cloudiness"] ==0)].dropna()
narrow_city_df                             

,City ID,City,Lat,lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
42,42,port blair,11.6667,92.7500,78.93,74,0,8.12,IN,1612659439
106,106,gualeguay,-33.1416,-59.3097,72.82,42,0,6.80,AR,1612659445
264,264,jalingo,8.8833,11.3667,74.95,20,0,5.06,NG,1612659461
320,320,te anau,-45.4167,167.7167,75.29,39,0,4.38,NZ,1612659466
345,345,yaan,7.3833,8.5667,76.41,21,0,5.08,NG,1612659469
410,410,nueve de julio,-35.4444,-60.8831,73.99,64,0,8.25,AR,1612659476
431,431,hambantota,6.1241,81.1185,71.60,100,0,3.44,LK,1612659478
538,538,poum,-20.2333,164.0167,79.99,68,0,4.83,NC,1612659489
549,549,caborca,30.5838,-111.1010,73.00,18,0,3.00,MX,1612659490
572,572,chichigalpa,12.5776,-87.0270,75.74,82,0,3.58,NI,1612659492


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [6]:
hotel_df = narrow_city_df.loc[:,["City","Country", "Lat", "lng", "Max Temp"]]

hotel_df["Hotel Name"] = ""

hotel_df


,City,Country,Lat,lng,Max Temp,Hotel Name
42,port blair,IN,11.6667,92.7500,78.93,
106,gualeguay,AR,-33.1416,-59.3097,72.82,
264,jalingo,NG,8.8833,11.3667,74.95,
320,te anau,NZ,-45.4167,167.7167,75.29,
345,yaan,NG,7.3833,8.5667,76.41,
410,nueve de julio,AR,-35.4444,-60.8831,73.99,
431,hambantota,LK,6.1241,81.1185,71.60,
538,poum,NC,-20.2333,164.0167,79.99,
549,caborca,MX,30.5838,-111.1010,73.00,
572,chichigalpa,NI,12.5776,-87.0270,75.74,


In [7]:
endpoint_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

params = {"type" : "hotel",
          "keyword" : "hotel",
          "radius" : 5000,
          "key" : g_key}

In [8]:

for index, row in hotel_df.iterrows():
    # get city name, lat, lnt from hotel_df
    lat = row["Lat"]
    lng = row["lng"]
    city_name = row["City"]
    
    # add keyword to params dict
    params["location"] = f"{lat},{lng}"

    # assemble url and make API request
    print(f"Getting the Results for Index {index}: '{city_name}'.")
    response = requests.get(endpoint_url, params=params).json()
    
    # extract results
    results = response['results']
    
    # save the hotel name to dataframe
    try:
        print(f"Closest hotel in '{city_name}' is {results[0]['name']}.")
        hotel_df.loc[index, "Hotel Name"] = results[0]['name']

    # if there is no hotel available, show missing field
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        
    print("------------")
    
    # Wait 1 sec to make another api request to avoid SSL Error
    #time.sleep(1)

# Print end of search once searching is completed
print("-------End of Search-------")

Getting the Results for Index 42: 'port blair'.
Closest hotel in 'port blair' is Symphony Samudra, Port Blair.
------------
Getting the Results for Index 106: 'gualeguay'.
Closest hotel in 'gualeguay' is Cabañas San Jorge.
------------
Getting the Results for Index 264: 'jalingo'.
Closest hotel in 'jalingo' is Shield Hotel.
------------
Getting the Results for Index 320: 'te anau'.
Closest hotel in 'te anau' is Lakefront Lodge Te Anau.
------------
Getting the Results for Index 345: 'yaan'.
Missing field/result... skipping.
------------
Getting the Results for Index 410: 'nueve de julio'.
Closest hotel in 'nueve de julio' is Grand Hotel Libertad - GHL.
------------
Getting the Results for Index 431: 'hambantota'.
Closest hotel in 'hambantota' is The Peacock Beach Hotel Hambantota.
------------
Getting the Results for Index 538: 'poum'.
Missing field/result... skipping.
------------
Getting the Results for Index 549: 'caborca'.
Closest hotel in 'caborca' is Hotel La Villa.
------------


In [9]:
selected_df = hotel_df.head(10)
selected_df

,City,Country,Lat,lng,Max Temp,Hotel Name
42,port blair,IN,11.6667,92.7500,78.93,"Symphony Samudra, Port Blair"
106,gualeguay,AR,-33.1416,-59.3097,72.82,Cabañas San Jorge
264,jalingo,NG,8.8833,11.3667,74.95,Shield Hotel
320,te anau,NZ,-45.4167,167.7167,75.29,Lakefront Lodge Te Anau
345,yaan,NG,7.3833,8.5667,76.41,
410,nueve de julio,AR,-35.4444,-60.8831,73.99,Grand Hotel Libertad - GHL
431,hambantota,LK,6.1241,81.1185,71.60,The Peacock Beach Hotel Hambantota
538,poum,NC,-20.2333,164.0167,79.99,
549,caborca,MX,30.5838,-111.1010,73.00,Hotel La Villa
572,chichigalpa,NI,12.5776,-87.0270,75.74,


In [10]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in selected_df.iterrows()]
locations = selected_df[["Lat", "lng"]]

In [11]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content = hotel_info)
fig.add_layer(markers)
#plt.savefig("C:/Users/joeki/Desktop/python-api-challenge/output_data/heatmap.png")

# Display figure
fig


Figure(layout=FigureLayout(height='420px'))

<Figure size 432x288 with 0 Axes>